In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
from transformers import BertTokenizer,  BertConfig, TFBertModel
import tensorflow as tf
from tensorflow.keras import backend as K

## 数据加载

In [2]:
ls

relation_extraction2.ipynb*  relation_extraction.ipynb*


In [3]:
def load_data(path):
    text_list = []
    spo_list = []
    with open(path, encoding='utf-8') as json_file:
        for line in json_file:
            text_list.append(literal_eval(line)['text'])
            spo_list.append(literal_eval(line)['spo_list'])
    return text_list, spo_list
    
# path='../data百度关系抽取数据集/train_data.json'
path='../data/百度关系抽取数据集/experiment.json'
text_list, spo_list = load_data(path)
print(text_list[0], spo_list[0])
# val_path = '../data/百度关系抽取数据集/dev_data.json'
val_path = '../data/百度关系抽取数据集/experiment.json'
val_text_list, val_spo_list = load_data(path)

查尔斯·阿兰基斯（Charles Aránguiz），1989年4月17日出生于智利圣地亚哥，智利职业足球运动员，司职中场，效力于德国足球甲级联赛勒沃库森足球俱乐部 [{'predicate': '出生地', 'object_type': '地点', 'subject_type': '人物', 'object': '圣地亚哥', 'subject': '查尔斯·阿兰基斯'}, {'predicate': '出生日期', 'object_type': 'Date', 'subject_type': '人物', 'object': '1989年4月17日', 'subject': '查尔斯·阿兰基斯'}]


In [4]:
len(text_list)

6

In [5]:
def load_predicate(path):
    with open(path,'r', encoding='utf-8')  as f:
        predicate_list = [literal_eval(i)['predicate'] for i in f]
    p2id = {}
    id2p = {}
    data = list(set(predicate_list))
    for i in range(len(data)):
        p2id[data[i]] = i
        id2p[i] = data[i]
    return p2id, id2p
    
path = '../data/百度关系抽取数据集/all_50_schemas'
p2id, id2p = load_predicate(path)
print(p2id)
print(id2p)


{'所属专辑': 0, '出生地': 1, '字': 2, '作者': 3, '作曲': 4, '创始人': 5, '身高': 6, '首都': 7, '气候': 8, '嘉宾': 9, '出生日期': 10, '总部地点': 11, '成立日期': 12, '连载网站': 13, '歌手': 14, '作词': 15, '号': 16, '面积': 17, '注册资本': 18, '占地面积': 19, '人口数量': 20, '父亲': 21, '编剧': 22, '导演': 23, '国籍': 24, '民族': 25, '出品公司': 26, '朝代': 27, '主角': 28, '上映时间': 29, '妻子': 30, '海拔': 31, '董事长': 32, '官方语言': 33, '主演': 34, '专业代码': 35, '制片人': 36, '所在城市': 37, '祖籍': 38, '目': 39, '修业年限': 40, '改编自': 41, '主持人': 42, '出版社': 43, '毕业院校': 44, '邮政编码': 45, '母亲': 46, '丈夫': 47, '简称': 48}
{0: '所属专辑', 1: '出生地', 2: '字', 3: '作者', 4: '作曲', 5: '创始人', 6: '身高', 7: '首都', 8: '气候', 9: '嘉宾', 10: '出生日期', 11: '总部地点', 12: '成立日期', 13: '连载网站', 14: '歌手', 15: '作词', 16: '号', 17: '面积', 18: '注册资本', 19: '占地面积', 20: '人口数量', 21: '父亲', 22: '编剧', 23: '导演', 24: '国籍', 25: '民族', 26: '出品公司', 27: '朝代', 28: '主角', 29: '上映时间', 30: '妻子', 31: '海拔', 32: '董事长', 33: '官方语言', 34: '主演', 35: '专业代码', 36: '制片人', 37: '所在城市', 38: '祖籍', 39: '目', 40: '修业年限', 41: '改编自', 42: '主持人', 43: '出版社', 44: '毕业院校', 45: '邮政编

## 数据集处理

In [6]:
def proceed_data(text_list,spo_list,p2id,id2p,tokenizer,max_length):
    id_label = {}
    ct = len(text_list)
    input_ids = np.zeros((ct,max_length),dtype='int32')
    attention_mask = np.zeros((ct,max_length),dtype='int32')
    start_tokens = np.zeros((ct,max_length),dtype='int32')
    end_tokens = np.zeros((ct,max_length),dtype='int32')
    send_s_po = np.zeros((ct,2),dtype='int32')
    object_start_tokens = np.zeros((ct,max_length,len(p2id)),dtype='int32')
    object_end_tokens = np.zeros((ct,max_length,len(p2id)),dtype='int32')
    invalid_index = []
    for k in range(ct):
        context_k = text_list[k].lower().replace(' ','')
        enc_context = tokenizer.encode(context_k,max_length=max_length,padding='max_length',truncation=True) 
        if len(spo_list[k])==0:
            invalid_index.append(k)
            continue
        start = []
        end = []
        S_index = []
        for j in range(len(spo_list[k])):
            answers_text_k = spo_list[k][j]['subject'].lower().replace(' ','')
            chars = np.zeros((len(context_k)))
            index = context_k.find(answers_text_k)
            chars[index:index+len(answers_text_k)]=1
            offsets = []
            idx=0
            for t in enc_context[1:]:
                w = tokenizer.decode([t])
                if '#' in w and len(w)>1:
                    w = w.replace('#','')
                if w == '[UNK]':
                    w = '。'
                offsets.append((idx,idx+len(w)))
                
                idx += len(w)
            toks = []
            for i,(a,b) in enumerate(offsets):
                sm = np.sum(chars[a:b])
                if sm>0: 
                    toks.append(i) 
            input_ids[k,:len(enc_context)] = enc_context
            attention_mask[k,:len(enc_context)] = 1
            if len(toks)>0:
                start_tokens[k,toks[0]+1] = 1
                end_tokens[k,toks[-1]+1] = 1
                start.append(toks[0]+1)
                end.append(toks[-1]+1)
                S_index.append(j)
                #随机抽取可以作为负样本提高准确率（不认同）
        if len(start) > 0:
            start_np = np.array(start)
            end_np = np.array(end)
            start_ = np.random.choice(start_np)
            end_ = np.random.choice(end_np[end_np >= start_])
            send_s_po[k,0] = start_
            send_s_po[k,1] = end_
            s_index = start.index(start_)
            #随机选取object的首位，如果选取错误，则作为负样本
            if end_ == end[s_index]:
                for index in range(len(start)):
                    if start[index] == start_ and end[index] == end_:
                        object_text_k = spo_list[k][S_index[index]]['object'].lower().replace(' ','')
                        predicate = spo_list[k][S_index[index]]['predicate']
                        p_id = p2id[predicate]
                        chars = np.zeros((len(context_k)))
                        index = context_k.find(object_text_k)
                        chars[index:index+len(object_text_k)]=1
                        offsets = [] 
                        idx=0
                        for t in enc_context[1:]:
                            w = tokenizer.decode([t])
                            if '#' in w and len(w)>1:
                                w = w.replace('#','')
                            if w == '[UNK]':
                                w = '。'
                            offsets.append((idx,idx+len(w)))
                            idx += len(w)
                        toks = []
                        for i,(a,b) in enumerate(offsets):
                            sm = np.sum(chars[a:b])
                            if sm>0: 
                                toks.append(i) 
                        if len(toks)>0:
                            id_label[p_id] = predicate
                            object_start_tokens[k,toks[0]+1,p_id] = 1
                            object_end_tokens[k,toks[-1]+1,p_id] = 1
        else:
            invalid_index.append(k)
    input_ids = tf.constant(input_ids)
    attention_mask = tf.constant(attention_mask)
    start_tokens = tf.constant(start_tokens)
    end_tokens = tf.constant(end_tokens)
    send_s_po = tf.constant(send_s_po)
    object_start_tokens = tf.constant(object_start_tokens)
    object_end_tokens = tf.constant(object_end_tokens)
    # invalid_index = tf.constant(invalid_index)
    return input_ids, attention_mask, start_tokens, end_tokens, send_s_po, object_start_tokens, object_end_tokens, invalid_index, id_label

max_length = 128  
model_path = '../model_dirs/bert-base-chinese'  
tokenizer = BertTokenizer.from_pretrained(model_path)    
input_ids, attention_mask, start_tokens, end_tokens, send_s_po, object_start_tokens, object_end_tokens, invalid_index, id_label \
= proceed_data(text_list,spo_list,p2id,id2p,tokenizer,max_length)

In [7]:
val_inputs = tokenizer(text_list, max_length=max_length, padding='max_length', truncation=True, return_tensors='tf') 
val_input_ids, val_attention_mask = val_inputs['input_ids'], val_inputs['attention_mask']


In [8]:
class LayerNormalization(tf.keras.layers.Layer):
    """(Conditional) Layer Normalization
    hidden_*系列参数仅为有条件输入时(conditional=True)使用
    """
    def __init__(
        self,
        center=True,
        scale=True,
        epsilon=None,
        conditional=False,
        hidden_units=None,
        hidden_activation='linear',
        hidden_initializer='glorot_uniform',
        **kwargs):
        super(LayerNormalization, self).__init__(**kwargs)
        self.center = center
        self.scale = scale
        self.conditional = conditional
        self.hidden_units = hidden_units
        self.hidden_activation = tf.keras.activations.get(hidden_activation)
        self.hidden_initializer = tf.keras.initializers.get(hidden_initializer)
        self.epsilon = epsilon or 1e-12
        
    def compute_mask(self, inputs, mask=None):
        if self.conditional:
            masks = mask if mask is not None else []
            masks = [m[None] for m in masks if m is not None]
            if len(masks) == 0:
                return None
            else:
                return K.all(K.concatenate(masks, axis=0), axis=0)
        else:
            return mask
        
    def build(self, input_shape):
        super(LayerNormalization, self).build(input_shape)
        if self.conditional:
            shape = (input_shape[0][-1],)
        else:
            shape = (input_shape[-1],)
        if self.center:
            self.beta = self.add_weight(
                shape=shape, initializer='zeros', name='beta')
        if self.scale:
            self.gamma = self.add_weight(
                shape=shape, initializer='ones', name='gamma')
        if self.conditional:
            if self.hidden_units is not None:
                self.hidden_dense = tf.keras.layers.Dense(
                    units=self.hidden_units,
                    activation=self.hidden_activation,
                    use_bias=False,
                    kernel_initializer=self.hidden_initializer)
            if self.center:
                self.beta_dense = tf.keras.layers.Dense(
                    units=shape[0], use_bias=False, kernel_initializer='zeros')
            if self.scale:
                self.gamma_dense = tf.keras.layers.Dense(
                    units=shape[0], use_bias=False, kernel_initializer='zeros')

    def call(self, inputs):
        """如果是条件Layer Norm，则默认以list为输入，第二个是condition
        """
        if self.conditional:
            inputs, cond = inputs
            if self.hidden_units is not None:
                cond = self.hidden_dense(cond)
            for _ in range(K.ndim(inputs) - K.ndim(cond)):
                cond = K.expand_dims(cond, 1)
            if self.center:
                beta = self.beta_dense(cond) + self.beta
            if self.scale:
                gamma = self.gamma_dense(cond) + self.gamma
        else:
            if self.center:
                beta = self.beta
            if self.scale:
                gamma = self.gamma
        outputs = inputs
        if self.center:
            mean = K.mean(outputs, axis=-1, keepdims=True)
            outputs = outputs - mean
        if self.scale:
            variance = K.mean(K.square(outputs), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            outputs = outputs / std
            outputs = outputs * gamma
        if self.center:
            outputs = outputs + beta
        return outputs
        


In [9]:
def new_loss(true,pred):
    true = tf.cast(true,tf.float32)
    loss = K.sum(K.binary_crossentropy(true, pred))
    return loss

In [10]:
def extract_subject(inputs):
    """根据subject_ids从output中取出subject的向量表征
    """
    output, subject_ids = inputs
    start = tf.gather(output,subject_ids[:,0],axis=1,batch_dims=0)
    end = tf.gather(output,subject_ids[:,1],axis=1,batch_dims=0)
    subject = tf.keras.layers.Concatenate(axis=2)([start, end])
    return subject[:,0]
'''
   output.shape = (None,128,768)
   subjudec_ids.shape = (None,2)
   start.shape = (None,None,768)
   subject.shape = (None,None,1536)
   subject[:,0].shape = (None,1536)
   这一部分给出各个变量的shape应该一目了然
'''
   
def build_model_2(pretrained_path, MAX_LEN, p2id):
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    s_po_index =  tf.keras.layers.Input((2,), dtype=tf.int32)
    
    bert_model = TFBertModel.from_pretrained(pretrained_path, output_hidden_states=True)
    outputs = bert_model(ids, attention_mask=att)
    x, _, hidden_states  = outputs[:3]
    layer_1 = hidden_states[-1]
    start_logits = tf.keras.layers.Dense(1,activation = 'sigmoid')(layer_1)
    start_logits = tf.keras.layers.Lambda(lambda x: x**2)(start_logits)
    
    end_logits = tf.keras.layers.Dense(1,activation = 'sigmoid')(layer_1)
    end_logits = tf.keras.layers.Lambda(lambda x: x**2)(end_logits)
    
    subject_1 = extract_subject([layer_1,s_po_index])
    Normalization_1 = LayerNormalization(conditional=True)([layer_1, subject_1])
    
    op_out_put_start = tf.keras.layers.Dense(len(p2id),activation = 'sigmoid')(Normalization_1)
    op_out_put_start = tf.keras.layers.Lambda(lambda x: x**4)(op_out_put_start)
    
    op_out_put_end = tf.keras.layers.Dense(len(p2id),activation = 'sigmoid')(Normalization_1)
    op_out_put_end = tf.keras.layers.Lambda(lambda x: x**4)(op_out_put_end)
    
    model = tf.keras.models.Model(inputs=[ids, att, s_po_index], outputs=[start_logits, end_logits, op_out_put_start, op_out_put_end])
    model_2 = tf.keras.models.Model(inputs=[ids, att], outputs=[start_logits,end_logits])
    model_3 = tf.keras.models.Model(inputs=[ids, att, s_po_index], outputs=[op_out_put_start, op_out_put_end])
    return model, model_2, model_3


In [11]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, model_2, model_3, id2tag, val_spo_list, val_input_ids, val_attention_mask, tokenizer):
        super(Metrics, self).__init__()
        self.model_2 = model_2
        self.model_3 = model_3
        self.id2tag = id2tag
        self.val_input_ids = val_input_ids
        self.val_attention_mask = val_attention_mask
        self.val_spo_list = val_spo_list
        self.tokenizer = tokenizer
        
    def on_train_begin(self, logs=None):
        self.val_f1s = []
        self.best_val_f1 = 0
    
    def get_same_element_index(self,ob_list):
        return [i for (i, v) in enumerate(ob_list) if v == 1]
    
    def evaluate_data(self):
        question=[]
        answer=[]
        y1 = self.model_2.predict([self.val_input_ids,self.val_attention_mask])
        for i in range(len(y1[0])):
            for z in self.val_spo_list[i]:
                question.append((z['subject'][0],z['subject'][-1],z['predicate'],z['object'][0],z['object'][-1]))
            x_ = [self.tokenizer.decode([t]) for t in self.val_input_ids[i]]
            x1 = np.array(y1[0][i]>0.5,dtype='int32')
            x2 = np.array(y1[1][i]>0.5,dtype='int32')
            union = x1 + x2
            index_list = self.get_same_element_index(list(union))
            start = 0
            S_list=[]
            while start+1 < len(index_list):
                S_list.append((index_list[start], index_list[start+1]+1))
                start += 2
            for os_s,os_e in S_list:
                S = ''.join(x_[os_s:os_e])
                s_e = tf.constant(np.array([[os_s,os_e]]))
                Y2 = self.model_3.predict([self.val_input_ids[i:i+1], self.val_attention_mask[i:i+1], s_e]) 
                for m in range(len(self.id2tag)):
                    x3 = np.array(Y2[0][0][:,m]>0.4,dtype='int32')
                    x4 = np.array(Y2[1][0][:,m]>0.4,dtype='int32')
                    if sum(x3)>0 and sum(x4)>0:
                        predict = self.id2tag[m]
                        union = x3 + x4
                        index_list = self.get_same_element_index(list(union))
                        start = 0
                        P_list=[]
                        while start+1 < len(index_list):
                            P_list.append((index_list[start],index_list[start+1]+1))
                            start += 2
                        for os_s,os_e in P_list:
                            if os_e>=os_s:
                                P = ''.join(x_[os_s:os_e])
                                answer.append((S[0],S[-1],predict,P[0],P[-1]))
        Q = set(question)
        S = set(answer)
        f1 = 2*len(Q&S)/(len(Q)+len(S))
        return f1
    
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        _val_f1 = self.evaluate_data()
        self.val_f1s.append(_val_f1)
        logs['val_f1'] = _val_f1
        if _val_f1 > self.best_val_f1:
            self.model.save_weights(f'../model_dirs/fine_tune_relation_extraction/{_val_f1}_tf_model.h5')
            self.best_val_f1 = _val_f1
            print("best f1: {}".format(self.best_val_f1))
        else:
            self.model.save_weights(f'../model_dirs/fine_tune_relation_extraction/{_val_f1}_tf_model.h5')
            print("val f1: {}, but not the best f1".format(_val_f1))
        return 

In [12]:
MAX_LEN = 128
# config = BertConfig.from_json_file('../model_dirs/bert-base-chinese/config.json')
# TFBertModel.from_pretrained(pretrained_path, config=config)
K.clear_session()
model,model_2,model_3 = build_model_2(model_path,  MAX_LEN, p2id)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(loss={'lambda': new_loss,
                'lambda_1': new_loss,
                'lambda_2': new_loss,
                'lambda_3': new_loss},optimizer=optimizer)
model.fit([input_ids, attention_mask, send_s_po],\
          [start_tokens,end_tokens,object_start_tokens,object_end_tokens], \
        epochs=3, batch_size=8, callbacks=[Metrics(model_2, model_3 ,id2p, val_spo_list,val_input_ids,val_attention_mask,tokenizer)])


Some layers from the model checkpoint at ../model_dirs/bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ../model_dirs/bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and

NotImplementedError: 

In [27]:
import os 
pid = os.getpid()
!kill -9 $pid